In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
r"""
@DATE: 2024-11-15 17:22:03
@File: test_fuc/vdktest.ipynb
@IDE: vscode
@Description:
    VikingDB 连接测试
"""

'\n@DATE: 2024-11-15 17:22:03\n@File: test_fuc/vdktest.ipynb\n@IDE: vscode\n@Description:\n    VikingDB 连接测试\n'

In [ ]:
from volcengine.viking_db import *
#host中不需要加http或者https等前缀
#当使用http时，host可以填写ip或域名, 当使用https时, host只能填写域名
#scheme 可选，可选值：http/https，默认http。
#connection_timeout，可配置，默认30s。
#socket_timeout，可配置，默认30s。
vikingdb_service = VikingDBService(host="api-vikingdb.volces.com", region="cn-beijing",connection_timeout=100)


In [ ]:
fields = [
    Field(
        field_name="img_path",
        field_type=FieldType.String,
        is_primary_key=True
    ),
    Field(
        field_name="image_vector",
        field_type=FieldType.Vector,
        dim=1024
    ),
]
res = vikingdb_service.create_collection("cupshe111502", fields, "This is an cupshe 1115 example")
print(res)

# 图片数据处理

In [7]:
import os
import csv
import base64
from PIL import Image 

def process_images(root_dir, output_file):
    """
    Processes images from a directory and generates a CSV file with id, path, and label for each image.

    Args:
        root_dir: The root directory containing the image folders.
        output_file: The path to the output CSV file.
    """

    if not os.path.exists(root_dir):
        print(f"Error: Directory '{root_dir}' does not exist.")
        return

    image_data = []
    image_id = 0

    for dirpath, dirnames, filenames in os.walk(root_dir):
        label = os.path.basename(dirpath)
        for filename in filenames:
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')): # 支持更多图片格式
                filepath = os.path.join(dirpath, filename)
                relative_path = os.path.relpath(filepath, root_dir)
                
                try:
                    with open(filepath, "rb") as image_file:
                        encoded_string = base64.b64encode(image_file.read()).decode('utf-8') # 解码为utf-8字符串
                        image_data.append([image_id, relative_path, label, encoded_string])
                except Exception as e:
                    print(f"Error encoding image {filepath}: {e}")
                    # 你可以选择跳过错误的图片，或者采取其他处理方式
                    continue
                image_id += 1

    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['id', 'path', 'label'])
        writer.writerows(image_data)

    print(f"CSV file '{output_file}' created successfully.")


if __name__ == "__main__":
    root_directory = "../test_img/raw_img"  # 请替换成你的实际路径
    output_csv_file = "image_data.csv"  # 请替换成你想要的输出文件名
    process_images(root_directory, output_csv_file)


CSV file 'image_data.csv' created successfully.


# 图片数据读取

In [ ]:
import csv
import sys

def process_csv(csv_filepath):
    """读取 CSV 文件，处理第四列 Base64 数据，并添加到 RawData 对象列表中。"""
    raw_data_list = []
    try:
        # 增加 CSV 字段大小限制
        maxInt = sys.maxsize
        while True:
            try:
                csv.field_size_limit(maxInt)
                break
            except OverflowError:
                maxInt = int(maxInt / 10)
        with open(csv_filepath, 'r', newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            next(reader)  # 跳过表头行 (如果存在)

            for row in reader:
                try:
                    base64_data = row[3]  # 获取第四列数据
                    # 创建 RawData 对象
                    raw_data_list.append(RawData("image", image=base64_data))
                except IndexError:
                    print(f"警告: 行 {reader.line_num} 数据不足，跳过。")
                except Exception as e:
                    print(f"错误: 处理行 {reader.line_num} 时出错: {e}")

        return raw_data_list
    except FileNotFoundError:
        print(f"错误: 文件 {csv_filepath} 未找到。")
        return None
    except Exception as e:
        print(f"错误: 读取或处理 CSV 文件时出错: {e}")
        return None



if __name__ == "__main__":
    csv_file_path = "image_data.csv"  # 请替换为您的 CSV 文件路径
    raw_data_list = process_csv(csv_file_path)

    if raw_data_list:
        print("RawData 对象列表:")
        print(raw_data_list)
        for raw_data in raw_data_list:
            print(raw_data)


{'sentence_dense_embedding': [[0.015674015507102013, 0.02749304473400116, -0.06347902864217758, 0.007382701151072979, -0.02070377580821514, -0.04909740388393402, -0.014267219230532646, 0.06227412819862366, -0.03802363574504852, 0.003132537007331848, 0.007263792213052511, 0.003085503587499261, -0.01805863156914711, -0.02351560816168785, 0.020236006006598473, 0.013886733911931515, -0.016405364498496056, -0.038069505244493484, 0.06933318078517914, -0.001820881967432797, 0.007601833436638117, 0.021111175417900085, -0.010281083174049854, 0.0033528332132846117, -0.02103748358786106, 0.017383666709065437, 0.026726627722382545, 0.04765533283352852, -0.01576811820268631, 0.03281315788626671, 0.010919786989688873, 0.03811078891158104, 0.06060216203331947, -0.011329017579555511, -0.017799369990825653, -0.02673904225230217, 0.011998816393315792, -0.003457835642620921, -0.005514266435056925, 0.02757546491920948, 0.004270684905350208, 0.001119886408559978, 0.02119535207748413, -0.0006762139964848757

In [31]:
list = [RawData("text", "hello1"), RawData("text", "hello2")]
res = vikingdb_service.embedding_v2(EmbModel("bge-m3", params={"return_token_usage": True}), list)
print(res)
print(res['token_usage'])

{'sentence_dense_embedding': [[-0.03546142578125, 0.051025390625, -0.040679931640625, -0.005092620849609375, -0.00020837783813476562, -0.031280517578125, -0.038177490234375, -0.0168914794921875, 0.003513336181640625, -0.0006346702575683594, -0.0182037353515625, -0.01055908203125, 0.0103759765625, -0.039459228515625, 0.02301025390625, -0.018646240234375, 0.0186004638671875, -0.026275634765625, 0.008026123046875, -0.02679443359375, -0.045928955078125, -0.0250701904296875, 0.00312042236328125, -0.006183624267578125, -0.00492095947265625, 0.027496337890625, -0.0391845703125, -0.015838623046875, -0.004566192626953125, -0.00797271728515625, 0.03656005859375, 0.044891357421875, -0.00936126708984375, -0.0692138671875, -0.0199127197265625, -0.02679443359375, 0.0253448486328125, -0.021087646484375, -0.07550048828125, 0.029022216796875, 0.020111083984375, 0.0240325927734375, 0.01251983642578125, -0.04437255859375, 0.0303955078125, -0.0250701904296875, -0.01513671875, 0.00051116943359375, -0.00074

# 图片向量化

In [67]:
res = vikingdb_service.embedding_v2(EmbModel("bge-visualized-m3", params={"return_token_usage": True}), raw_data_list)
print(res)
print(res['sentence_dense_embedding'][0])
print(res['token_usage'])

{'sentence_dense_embedding': [[0.01684974879026413, 0.027372701093554497, -0.06391550600528717, 0.007171706762164831, -0.021440140902996063, -0.0495990626513958, -0.014043721370398998, 0.06093720346689224, -0.03836401551961899, 0.003206147812306881, 0.007085667457431555, 0.0029289277736097574, -0.017908746376633644, -0.023557955399155617, 0.020558731630444527, 0.013543828390538692, -0.016034986823797226, -0.03889353573322296, 0.06920973211526871, -0.0017082508420571685, 0.007771594449877739, 0.02190633863210678, -0.00964740663766861, 0.0033938803244382143, -0.021028466522693634, 0.017508869990706444, 0.02717774361371994, 0.04827117919921875, -0.015392781235277653, 0.032954588532447815, 0.010623076930642128, 0.03814798966050148, 0.06118866056203842, -0.011188426055014133, -0.01865105889737606, -0.02726295217871666, 0.012439876794815063, -0.00372614455409348, -0.005171032622456551, 0.028492722660303116, 0.004074929282069206, 0.00034011516254395247, 0.021672822535037994, -8.61158478073775

# 数据写入

In [38]:
# 获取指定数据集，程序初始化时调用即可，无需重复调用
collection = vikingdb_service.get_collection("cupshe111502")

In [ ]:
sentence1 = res['sentence_dense_embedding'][0]
sentence2 = res['sentence_dense_embedding'][1]

field1 = {"img_path": "/Users/bytedance/Documents/workproject/Eimage-search/test_fuc/test_img/test1.jpg","text_vector":sentence1}
field2 = {"img_path": "/Users/bytedance/Documents/workproject/Eimage-search/test_fuc/test_img/test2.jpg","text_vector":sentence2}

data1 = Data(field1)
data2 = Data(field2)
datas = []
datas.append(data1)
datas.append(data2)
collection.upsert_data(datas)  


In [68]:
resfetch= collection.fetch_data("/Users/bytedance/Documents/workproject/Eimage-search/test_fuc/test_img/test1.jpg")


In [69]:
resfetch.fields

{'img_path': '/Users/bytedance/Documents/workproject/Eimage-search/test_fuc/test_img/test1.jpg',
 'text_vector': [0.01684974879026413,
  0.027372701093554497,
  -0.06391550600528717,
  0.007171706762164831,
  -0.021440140902996063,
  -0.0495990626513958,
  -0.014043721370398998,
  0.06093720346689224,
  -0.03836401551961899,
  0.003206147812306881,
  0.007085667457431555,
  0.0029289277736097574,
  -0.017908746376633644,
  -0.023557955399155617,
  0.020558731630444527,
  0.013543828390538692,
  -0.016034986823797226,
  -0.03889353573322296,
  0.06920973211526871,
  -0.0017082508420571685,
  0.007771594449877739,
  0.02190633863210678,
  -0.00964740663766861,
  0.0033938803244382143,
  -0.021028466522693634,
  0.017508869990706444,
  0.02717774361371994,
  0.04827117919921875,
  -0.015392781235277653,
  0.032954588532447815,
  0.010623076930642128,
  0.03814798966050148,
  0.06118866056203842,
  -0.011188426055014133,
  -0.01865105889737606,
  -0.02726295217871666,
  0.01243987679481506

In [70]:
datas = []
for i in range(len(raw_data_list)):
    sentence = res['sentence_dense_embedding'][i]
    field = {"img_path": "/Users/bytedance/Documents/workproject/Eimage-search/test_fuc/test_img/test1.jpg","text_vector":sentence}
    data = Data(field)
    datas.append(data)
    
collection.upsert_data(datas)  
